https://www.motorola.com/blog/en-us/all-post-page

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import sys,bs4,re,time,requests,pandas as pd
from bs4 import BeautifulSoup

import warnings
warnings.filterwarnings("ignore")

%autosave 1

Autosaving every 1 seconds


In [2]:
#driver for operation
from webdriver_manager.chrome import ChromeDriverManager
option = webdriver.ChromeOptions()
option.add_argument('headless')
driver = webdriver.Chrome(ChromeDriverManager().install()
#                           ,options=option
                         )



====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/100.0.4896.60/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\AKSHAY SATPUTE\.wdm\drivers\chromedriver\win32\100.0.4896.60]


In [3]:
def parse_webpage_bs(search_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0"}
    try:
        site_request = requests.get(search_url, headers=headers, timeout=10)
    except requests.exceptions.RequestException as e:
        print(e)
        site_request = None
    if site_request != None and site_request.status_code==200:
        site_soup = bs4.BeautifulSoup(site_request.content, "lxml")
    else:
        site_soup = None
    return site_soup

In [4]:


driver.maximize_window()

url = f'https://www.motorola.com/blog/en-us/all-post-page'

driver.get(url)

driver.find_element_by_xpath('''/html/body/div[3]/div/div/div/div[2]/div/div[1]''').click()

driver.refresh()

hrefs = []

range_count = 3

try:
    for x in range(range_count):
        time.sleep(3)
        driver.find_element_by_xpath('''/html/body/div[2]/div/div[1]/div/div/div[2]/div/div/div/div[4]/div/a[2]''').click()

        for y in range(11):
            
            links = driver.find_elements_by_xpath(f'''/html/body/div[2]/div/div[1]/div/div/div[2]/div/div/div/div[3]/div[{y}]/a''')
            for l in links:
                hrefs.append(l.get_attribute('href'))

        print(str(range_count-(x+1)) + " pages remaining..",end="\r")
except:
    pass

In [5]:


articles  = []
dates = []
titles = []
thumbnails = []

try:
    for number,link in enumerate(hrefs):

        driver.get(link)
        time.sleep(1)
        driver.implicitly_wait(100)
        title = driver.find_element_by_xpath('''/html/body/div[2]/div/div[1]/div/div/div[2]/div/div/div[1]/div[3]/div[2]/div/h1''').text
        titles.append(title)
        article = driver.find_element_by_xpath('''/html/body/div[2]/div/div[1]/div/div/div[2]/div/div/div[2]''').text
        articles.append(article)
        date = driver.find_element_by_xpath('''/html/body/div[2]/div/div[1]/div/div/div[2]/div/div/div[1]/div[3]/div[4]/div/span[2]''').text
        dates.append(date)
        thumbnail = driver.find_element_by_xpath('''/html/body/div[2]/div/div[1]/div/div/div[2]/div/div/div[1]/div[1]/img''').get_attribute('src')
        thumbnails.append(thumbnail)
        print(str(len(hrefs)-(number+1)) + " url remaining..",end="\r")


    print()
    
except:
    pass

0 url remaining...


In [6]:
#zippling the list so that it'll form the tuple
zipped = list(zip(dates,titles,articles,hrefs,thumbnails))
zipped[:3]

[('APRIL 29, 2022',
  'Motorola’s most advanced camera system ever introduces instant all pixel-focus',
  'With the motorola edge 30 pro we introduced our most advanced camera system ever. From the two 50 MP cameras, which deliver ultra-wide angle shots and extreme close-ups, to 8K video recording and the highest video resolution possible on a smartphone today, the motorola edge 30 pro captures every incredible detail. You’ll also be able to eliminate unwanted shakes with OIS, which automatically compensates against blurred images and videos due to unwanted camera movements, so even when an exposure takes longer in low light conditions — clarity and detail shine through. \nBut perhaps most exciting is that the motorola edge 30 pro comes complete with Instant All Pixel Focus - Omni PDAF - our first ever on a Motorola device. And the ability to shoot breathtaking videos that meet strict HDR10+ standards for color accuracy, color range, brightness and contrast. See how you get 32x more fo

In [7]:
#creating dataframe 
temp_df = pd.DataFrame(zipped,columns=['date','title','article','url','thumbnail'])
temp_df.head()

,date,title,article,url,thumbnail
0,"APRIL 29, 2022",Motorola’s most advanced camera system ever in...,With the motorola edge 30 pro we introduced ou...,https://www.motorola.com/blog/post?id=441,https://motorolablog.myvtex.com/arquivos/camer...
1,"APRIL 27, 2022",Say hello to the thinnest 5G smartphone around...,Motorola is introducing today the newest addit...,https://www.motorola.com/blog/post?id=435,https://motorolablog.myvtex.com/arquivos/edge%...
2,"JANUARY 28, 2022",Motorola’s top tech predictions for 2022,"As we welcome in another new year, the possibi...",https://www.motorola.com/blog/post?id=389,https://motorolablog.myvtex.com/arquivos/Movem...
3,"DECEMBER 21, 2021",Android 12 + My UX: More personal and effortle...,"At Motorola, we believe consumers deserve the ...",https://www.motorola.com/blog/post?id=387,https://motorolablog.myvtex.com/arquivos/Andro...
4,"DECEMBER 16, 2021","The Power of Design, Meet Lewis Suaton",Lewis Suaton is a fashion designer based out o...,https://www.motorola.com/blog/post?id=386,https://motorolablog.myvtex.com/arquivos/lewis...


In [8]:
#to csv
temp_df.to_csv('moto news.csv')

#to json
temp_df.to_json('moto news.json')